In [27]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label

def generate_landscape(grid_size=(50, 50), terrain_types=5):
    """生成隨機地形 (2D grid)。"""
    return np.random.randint(0, terrain_types, size=grid_size)

def visualize_landscape(landscape, filename):
    """視覺化並存檔。"""
    plt.figure(figsize=(6, 6))
    plt.imshow(landscape, cmap='terrain', origin='upper')
    plt.colorbar(label="Terrain Type")
    plt.axis('off')
    plt.savefig(filename, bbox_inches='tight')
    plt.close()

def calculate_region_match(landscape, region_coords, target_terrain):
    """
    計算指定區域中，目標地形 (target_terrain) 的覆蓋比例。
    region_coords = (row_start, row_end, col_start, col_end)
    """
    region = landscape[region_coords[0]:region_coords[1], region_coords[2]:region_coords[3]]
    match_score = np.sum(region == target_terrain) / region.size
    return match_score

def calculate_river_connectivity(landscape, river_type=0, min_length=7):
    """
    檢查是否存在任意一個連通的河流，其長度(像素數量)超過 min_length。
    回傳值:
      1.0 若有任意河流區塊 >= min_length
      0.0 否則
    """
    labeled_array, num_features = label(landscape == river_type)
    if num_features == 0:
        return 0.0  # 沒有任何河流像素
    
    # 計算各個連通區的大小
    sizes = np.bincount(labeled_array.ravel())[1:] 
    # 如果有任何一個區塊的大小 >= min_length
    has_long_river = np.any(sizes >= min_length)
    return 1.0 if has_long_river else 0.0

def fitness_function(landscape, terrain_types=5):
    """
    綜合適應度函數:
    1) 北部是否為雪山 (type=4)
    2) 南部是否為草原 (type=1)
    3) 是否有長度 >= 7 的河流 (type=0)
    """
    rows, cols = landscape.shape

    # 1) 北部雪山評分
    north_region_coords = (0, rows // 3, 0, cols)
    north_match_score = calculate_region_match(landscape, north_region_coords, target_terrain=4)

    # 2) 南部草原評分
    south_region_coords = (2 * rows // 3, rows, 0, cols)
    south_match_score = calculate_region_match(landscape, south_region_coords, target_terrain=1)

    # 3) 河流「足夠長」評分 (有就 1, 沒有就 0)
    river_connectivity_score = calculate_river_connectivity(landscape, river_type=0, min_length=7)

    # 加權
    total_fitness = (
        0.4 * north_match_score +
        0.4 * south_match_score +
        0.2 * river_connectivity_score
    )

    return total_fitness

def crossover(parent1, parent2):
    """遮罩交配: 隨機將 parent1 或 parent2 的像素拿來生成 child。"""
    mask = np.random.randint(0, 2, size=parent1.shape).astype(bool)
    child = np.where(mask, parent1, parent2)
    return child

def mutate(landscape, mutation_rate=0.01, terrain_types=5):
    """隨機突變: 以 mutation_rate 的機率，將像素替換為隨機地形。"""
    mutation_mask = np.random.rand(*landscape.shape) < mutation_rate
    new_values = np.random.randint(0, terrain_types, size=landscape.shape)
    landscape[mutation_mask] = new_values[mutation_mask]
    return landscape

def initialize_population(pop_size, grid_size, terrain_types):
    """初始化種群: 生成 pop_size 張隨機地形圖。"""
    return [generate_landscape(grid_size, terrain_types) for _ in range(pop_size)]

def evolve_population(population, fitnesses, terrain_types, mutation_rate):
    """進化過程: 選擇、交配 (crossover) 及突變 (mutation)。"""
    
    sorted_indices = np.argsort(fitnesses)[::-1]  
    top_half = [population[i] for i in sorted_indices[:len(population) // 2]]

  
    new_population = []
    while len(new_population) < len(population):
        parents = np.random.choice(len(top_half), size=2, replace=False)
        parent1, parent2 = top_half[parents[0]], top_half[parents[1]]
        child = crossover(parent1, parent2)
        child = mutate(child, mutation_rate, terrain_types)
        new_population.append(child)

    return new_population

def main_evolution(
    output_folder='output',
    num_generations=300,     
    pop_size=1000,             
    grid_size=(50, 50),
    terrain_types=5,
    mutation_rate=0.02        
):
    
    os.makedirs(output_folder, exist_ok=True)

    
    population = initialize_population(pop_size, grid_size, terrain_types)

    for generation in range(num_generations):
       
        fitnesses = [fitness_function(ind, terrain_types) for ind in population]
        best_fitness = max(fitnesses)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness:.3f}")

        
        if generation >= num_generations - 10:
            best_index = np.argmax(fitnesses)
            best_landscape = population[best_index]
            filename = os.path.join(
                output_folder,
                f'best_landscape_gen_{generation+1}_fitness_{fitnesses[best_index]:.3f}.png'
            )
            visualize_landscape(best_landscape, filename)

        
        population = evolve_population(population, fitnesses, terrain_types, mutation_rate)

    print("Evolution Complete!")

if __name__ == '__main__':
    main_evolution()


Generation 1: Best Fitness = 0.385
Generation 2: Best Fitness = 0.390
Generation 3: Best Fitness = 0.391
Generation 4: Best Fitness = 0.396
Generation 5: Best Fitness = 0.401
Generation 6: Best Fitness = 0.405
Generation 7: Best Fitness = 0.411
Generation 8: Best Fitness = 0.416
Generation 9: Best Fitness = 0.421
Generation 10: Best Fitness = 0.426
Generation 11: Best Fitness = 0.427
Generation 12: Best Fitness = 0.430
Generation 13: Best Fitness = 0.437
Generation 14: Best Fitness = 0.442
Generation 15: Best Fitness = 0.445
Generation 16: Best Fitness = 0.454
Generation 17: Best Fitness = 0.456
Generation 18: Best Fitness = 0.457
Generation 19: Best Fitness = 0.463
Generation 20: Best Fitness = 0.464
Generation 21: Best Fitness = 0.469
Generation 22: Best Fitness = 0.474
Generation 23: Best Fitness = 0.474
Generation 24: Best Fitness = 0.480
Generation 25: Best Fitness = 0.481
Generation 26: Best Fitness = 0.487
Generation 27: Best Fitness = 0.491
Generation 28: Best Fitness = 0.493
G